*This notebook contains an excerpt from the [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) by Jake VanderPlas; the content is available [on GitHub](https://github.com/jakevdp/PythonDataScienceHandbook).*

# Hierarchical Indexing

In [1]:
import pandas as pd
import numpy as np

In [2]:
population_data = [{'state': 'California', 'year': '2000', 'population': 33871648},
 {'state': 'California', 'year': '2010', 'population': 37253956},
 {'state': 'New York', 'year': '2000', 'population': 18976457},
 {'state': 'New York', 'year': '2010', 'population': 19378102},
 {'state': 'Texas', 'year': '2000', 'population': 20851820},
 {'state': 'Texas', 'year': '2010', 'population': 25145561}]

In [3]:
pop_df = pd.DataFrame(population_data)
pop_df

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


Select records of California

In [4]:
pop_df[pop_df['state']=='California']

,state,year,population
0,California,2000,33871648
1,California,2010,37253956


In [5]:
pop_df2 = pop_df.set_index('state')
pop_df2

,year,population
state,,
California,2000,33871648
California,2010,37253956
New York,2000,18976457
New York,2010,19378102
Texas,2000,20851820
Texas,2010,25145561


In [6]:
pop_df2.loc['California']

,year,population
state,,
California,2000,33871648
California,2010,37253956


In [7]:
pop_df2.reset_index()

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


What is the population of California in year 2000?

In [8]:
q = pop_df[(pop_df['year']=='2000') & (pop_df['state']=='California')]
q

,state,year,population
0,California,2000,33871648


In [9]:
q.loc[0, 'population']

33871648

In [10]:
pop_df2

,year,population
state,,
California,2000,33871648
California,2010,37253956
New York,2000,18976457
New York,2010,19378102
Texas,2000,20851820
Texas,2010,25145561


In [11]:
pop_df2[pop_df2['year']=='2000'].loc['California', 'population']

33871648

In [12]:
pop_df

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [13]:
pop_df3 = pop_df.set_index(['state', 'year'])
pop_df3

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

In [14]:
pop_df3.index

MultiIndex([('California', '2000'),
            ('California', '2010'),
            (  'New York', '2000'),
            (  'New York', '2010'),
            (     'Texas', '2000'),
            (     'Texas', '2010')],
           names=['state', 'year'])

In [15]:
pop_df3.loc['California']

,population
year,
2000,33871648
2010,37253956


In [16]:
pop_df3.loc[('California', '2010')]

population    37253956
Name: (California, 2010), dtype: int64

In [17]:
pop_df3.loc[(:, '2010')]

SyntaxError: invalid syntax (<ipython-input-17-cf78904ad9b7>, line 1)

In [18]:
pop_df3

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

In [19]:
idx = pd.IndexSlice
pop_df3.loc[idx[:, '2010'], :]

,,population
state,year,
California,2010,37253956
New York,2010,19378102
Texas,2010,25145561


### MultiIndex as extra dimension

You might notice something else here: we could easily have stored the same data using a simple ``DataFrame`` with index and column labels.
In fact, Pandas is built with this equivalence in mind. The ``unstack()`` method will quickly convert a multiply indexed ``Series`` into a conventionally indexed ``DataFrame``:

In [20]:
pop_df3['under18'] = [9267089, 9284094,
                      4687374, 4318033,
                      5906301, 6879014]
pop_df3

population  under18
state      year                     
California 2000    33871648  9267089
           2010    37253956  9284094
New York   2000    18976457  4687374
           2010    19378102  4318033
Texas      2000    20851820  5906301
           2010    25145561  6879014

In [21]:
f_u18 = pop_df3['under18'] / pop_df3['population']
f_u18

state       year
California  2000    0.273594
            2010    0.249211
New York    2000    0.247010
            2010    0.222831
Texas       2000    0.283251
            2010    0.273568
dtype: float64

In [22]:
f_u18.unstack()

year,2000,2010
state,,
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


In [23]:
f_u18.unstack('state')

state,California,New York,Texas
year,,,
2000,0.273594,0.247010,0.283251
2010,0.249211,0.222831,0.273568


In [24]:
f_u18.unstack('state').stack()

year  state     
2000  California    0.273594
      New York      0.247010
      Texas         0.283251
2010  California    0.249211
      New York      0.222831
      Texas         0.273568
dtype: float64

# Using multi-index to calculate aggregations

In [26]:
f_u18

state       year
California  2000    0.273594
            2010    0.249211
New York    2000    0.247010
            2010    0.222831
Texas       2000    0.283251
            2010    0.273568
dtype: float64

In [27]:
f_u18.mean()

0.25824409535188225

In [28]:
f_u18.unstack('state')

state,California,New York,Texas
year,,,
2000,0.273594,0.247010,0.283251
2010,0.249211,0.222831,0.273568


In [29]:
f_u18.unstack('state').mean()

state
California    0.261403
New York      0.234920
Texas         0.278409
dtype: float64

In [30]:
f_u18.unstack('year').mean()

year
2000    0.267952
2010    0.248536
dtype: float64